In [1]:
from datasets import load_dataset

/Users/charlesoneill/miniconda3/envs/jsalt/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "charlieoneill/jsalt-astroph-dataset"
dataset = load_dataset(path)

In [3]:
# Get the entries where arxiv_id is 1607.02388
entries = dataset["train"].filter(lambda x: x["arxiv_id"] == "1607.02388")
print(entries)

Filter: 100%|██████████| 271544/271544 [00:06<00:00, 44600.38 examples/s]

Dataset({
    features: ['subfolder', 'filename', 'abstract', 'introduction', 'conclusions', 'year', 'month', 'arxiv_id'],
    num_rows: 1
})


In [5]:
entries['abstract']

[' ']

In [7]:
entries['conclusions']

['']

In [8]:
entries['introduction']

["The initial mass of a star on the zero-age main sequence largely determines its fate. Stars born with masses exceeding $\\sim 10\\,M_\\odot$ end their lives in a core-collapse event, e.g. a supernova (SN) explosion, and leave a neutron star or a black hole as remnant \\citep{Heger2003}. Such massive stars are luminous, with bolometric luminosities exceeding $L_{\\rm bol}\\msim 10^4\\,L_\\odot$. On the main sequence, massive stars have spectral types earlier than B2V. These bright stars live very fast, the most massive of them die within just $\\sim 10$\\,Myr. Albeit we see many massive stars by naked eye in the night sky (e.g.\\ the Orion Belt consists of massive stars), these stars are actually very rare and constitute only $\\sim 0.4$\\%\\ of all stars in our Milky Way.  Despite their small number, massive stars have enormous impact on the galactic ecology. Their strong ionizing radiation and stellar winds, as well as their final demise in  SN explosions, largely determine the phys

In [16]:
import feedparser
import urllib, urllib.request

def get_abstract(arxiv_id):

    query = arxiv_id
    url = 'http://export.arxiv.org/api/query?search_query='+query+'&start=0&max_results=1'
    data = urllib.request.urlopen(url)
    # print(data.read().decode('utf-8'))
    data = data.read().decode('utf-8')
    feed = feedparser.parse(data)
    return feed['entries'][0]['summary']

abstract = 'physics/9804027'
get_abstract(abstract)

'It is shown that the plasma, generated during an impact of a meteoroid with\nan artificial satellite, can produce electromagnetic radiation below the\nmicrowave frequency range. This interference is shown to exceed local noise\nsources and might disturb regular satellite operations.'

In [14]:
from datasets import load_dataset

ds = load_dataset("JSALT2024-Astro-LLMs/astro_paper_corpus")

Trying to resume download...
Trying to resume download...
Generating train split: 100%|██████████| 271544/271544 [00:04<00:00, 61724.03 examples/s]


In [34]:
path = "/Users/charlesoneill/retrieval/data/multi_paper_sentences.json"

import json

with open(path, 'r') as f:
    data = json.load(f)

In [35]:
data

{'1403.0007_1': {'title': 'Cosmic Star-Formation History',
  'question': 'Research question: How do parameter uncertainties and degeneracies in semianalytic models and subgrid implementations of galaxy formation affect the predictive power of cosmological simulations?',
  'text': 'The purely phenomenological treatment of complex physical processes that is at the core of semianalytic schemes of galaxy formation (  ) and—at a much higher level of realism—the “subgrid modeling” of star formation and stellar feedback that must be implemented even in the more accurate cosmological hydrodynamic simulations (      ) are sensitive to poorly determined parameters and suffer from various degeneracies, a weakness that has traditionally prevented robust predictions to be made in advance of specific observations.',
  'citations': [['Ocvirk', '2008'],
   ['White', 'Frenk', '1991'],
   ['Agertz', '2013'],
   ['Keres', '2005'],
   ['Katz', '1996'],
   ['Springel', 'Hernquist', '2003'],
   ['Governato'

In [37]:
dataset_path = "charlieoneill/jsalt-astroph-dataset"

dataset = load_dataset(dataset_path)

arxiv_ids = dataset['train']['arxiv_id']

In [39]:
arxiv_id = '0612.306'
entry = dataset['train'].filter(lambda x: x['arxiv_id'] == arxiv_id)
entry

Filter: 100%|██████████| 271544/271544 [00:01<00:00, 172969.42 examples/s]


Dataset({
    features: ['subfolder', 'filename', 'abstract', 'introduction', 'conclusions', 'year', 'month', 'arxiv_id'],
    num_rows: 0
})

In [40]:
arxiv_ids = dataset['train']['arxiv_id']

# Find closest match to arxiv_id
import difflib

arxiv_id = '0612306'
closest_match = difflib.get_close_matches(arxiv_id, arxiv_ids)
closest_match

['1306.2306', '1306.1230', '1012.3706']

In [43]:
import json
from datasets import load_dataset
from tqdm import tqdm

def filter_arxiv_ids(file_path, valid_arxiv_ids):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    total_removed_ids = 0
    total_removed_elements = 0
    filtered_data = {}

    for item_id, item in tqdm(data.items(), desc=f"Processing {file_path}"):
        original_length = len(item['arxiv'])
        item['arxiv'] = [arxiv_id for arxiv_id in item['arxiv'] if arxiv_id in valid_arxiv_ids]
        total_removed_ids += original_length - len(item['arxiv'])
        
        if item['arxiv']:  # Only keep elements with non-empty 'arxiv' lists
            filtered_data[item_id] = item
        else:
            total_removed_elements += 1
    
    with open(file_path, 'w') as f:
        json.dump(filtered_data, f, indent=2)
    
    return total_removed_ids, total_removed_elements

def main():
    # Load the jsalt-astro dataset
    dataset_path = "charlieoneill/jsalt-astroph-dataset"
    dataset = load_dataset(dataset_path)
    valid_arxiv_ids = set(dataset['train']['arxiv_id'])

    # Process multi_paper.json
    multi_paper_path = '../data/multi_paper.json'
    removed_ids_multi_paper, removed_elements_multi_paper = filter_arxiv_ids(multi_paper_path, valid_arxiv_ids)
    print(f"multi_paper.json:")
    print(f"  Removed {removed_ids_multi_paper} arXiv IDs")
    print(f"  Removed {removed_elements_multi_paper} elements with empty 'arxiv' lists")

    # Process multi_paper_sentences.json
    multi_paper_sentences_path = '../data/multi_paper_sentences.json'
    removed_ids_multi_paper_sentences, removed_elements_multi_paper_sentences = filter_arxiv_ids(multi_paper_sentences_path, valid_arxiv_ids)
    print(f"multi_paper_sentences.json:")
    print(f"  Removed {removed_ids_multi_paper_sentences} arXiv IDs")
    print(f"  Removed {removed_elements_multi_paper_sentences} elements with empty 'arxiv' lists")

main()

Processing ../data/multi_paper.json: 100%|██████████| 248/248 [00:00<00:00, 936262.28it/s]


multi_paper.json:
  Removed 0 arXiv IDs
  Removed 17 elements with empty 'arxiv' lists


Processing ../data/multi_paper_sentences.json: 100%|██████████| 574/574 [00:00<00:00, 999389.99it/s]

multi_paper_sentences.json:
  Removed 0 arXiv IDs
  Removed 86 elements with empty 'arxiv' lists


In [44]:
import json
from datasets import load_dataset
from tqdm import tqdm

def filter_top_250_multipapers(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Sort items by the length of their 'arxiv' list in descending order
    sorted_items = sorted(data.items(), key=lambda x: len(x[1]['arxiv']), reverse=True)

    # Keep only the top 250 items
    top_250_items = dict(sorted_items[:250])

    total_removed_elements = len(data) - len(top_250_items)

    with open(file_path, 'w') as f:
        json.dump(top_250_items, f, indent=2)

    return total_removed_elements

def main():
    # Process multi_paper_sentences.json
    multi_paper_sentences_path = '../data/multi_paper_sentences.json'
    removed_elements = filter_top_250_multipapers(multi_paper_sentences_path)
    print(f"multi_paper_sentences.json:")
    print(f"  Kept top 250 multipapers with largest arxiv lists")
    print(f"  Removed {removed_elements} elements")

if __name__ == "__main__":
    main()

multi_paper_sentences.json:
  Kept top 250 multipapers with largest arxiv lists
  Removed 238 elements


In [1]:
import wandb
import pandas as pd

# Replace with your project and entity (team) names
entity = 'charlieoneill'
project = 'jsalt-astro'

# Initialize API
api = wandb.Api()

# Get all runs in the project
runs = api.runs(f"{entity}/{project}")

# List to hold dataframes
dfs = []

# Loop through each run
for run in runs:
    print(run)
    # Get the run history as a dataframe
    history = run.history()
    # Add a column for the run ID to identify different runs later
    history['run_id'] = run.id
    # Get the name
    history['name'] = run.name
    # Append to list
    dfs.append(history)

# Concatenate all dataframes
all_data = pd.concat(dfs, ignore_index=True)

# Make name the index
#all_data.set_index('name', inplace=True)

# # Save to CSV or any other format you prefer
# all_data.to_csv('../data/wandb_project_data.csv', index=False)

# print("Data saved to wandb_project_data.csv")

<Run charlieoneill/jsalt-astro/rsyrba3l (finished)>
<Run charlieoneill/jsalt-astro/cykp8zsr (finished)>
<Run charlieoneill/jsalt-astro/xeo88sw2 (finished)>
<Run charlieoneill/jsalt-astro/fe7x43kd (finished)>


In [2]:
all_data

,multipaper_sentences_recall@100,multipaper_sentences_ndcg@100,multipaper_sentences_ndcg@50,multipaper_sentences_map@100,single_doc_success_rate,multipaper_sentences_map@10,multipaper_sentences_recall@10,multipaper_sentences_recall@50,multipaper_sentences_map@50,_timestamp,single_doc_reciprocal_rank,_runtime,single_doc_avg_precision,multipaper_sentences_ndcg@10,_step,run_id,name
0,0.392298,0.217216,0.185359,0.079675,0.840,0.054795,0.117146,0.291435,0.074433,1.719937e+09,0.735300,2003.990883,0.735300,0.113839,0,rsyrba3l,HyDECohereRerank
1,0.358187,0.198842,0.171545,0.071906,0.684,0.049526,0.108876,0.270544,0.067740,1.719942e+09,0.469678,1480.548532,0.469678,0.105853,0,cykp8zsr,BaseHyDE
2,0.197671,0.107686,0.092119,0.035213,0.460,0.026387,0.058327,0.146637,0.033537,1.719942e+09,0.289830,368.576570,0.289830,0.056603,0,xeo88sw2,BagOfWordsDateFiltered
3,0.315701,0.176014,0.150634,0.062287,0.704,0.043371,0.094868,0.234335,0.058628,1.719943e+09,0.497546,221.181892,0.497546,0.093872,0,fe7x43kd,BaseSemanticSearch


In [10]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

def create_horizontal_bar_plot(df, column_name, width=1200, height=600):
    # Sort the dataframe by the column values in descending order
    df_sorted = df.sort_values(by=column_name, ascending=True)
    
    # Create a uniformly spaced discrete color sequence
    n_bars = len(df_sorted)
    color_sequence = px.colors.sequential.Viridis
    colors = [color_sequence[i * len(color_sequence) // n_bars] for i in range(n_bars)]
    
    # Create the horizontal bar plot
    fig = go.Figure(go.Bar(
        y=df_sorted['name'],
        x=df_sorted[column_name],
        orientation='h',
        marker_color=colors,
        text=df_sorted[column_name].round(2),
        textposition='outside',
        textfont=dict(family="Palatino", size=18),
    ))
    
    # Determine the title based on the column name
    if "multipaper_sentences" in column_name:
        title = "Multi Paper"
    elif "single_doc" in column_name:
        title = "Single Paper"
    else:
        title = ""
    
    # Remove prefix from x-axis label
    x_axis_label = column_name.replace("multipaper_sentences_", "").replace("single_doc_", "")
    
    # Set the layout properties
    fig.update_layout(
        title={
            'text': title,
            'font': {'family': "Palatino", 'size': 36},
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis_title={
            'text': x_axis_label,
            'font': {'family': "Palatino", 'size': 28}
        },
        font=dict(family="Palatino", size=24),
        height=height,
        width=width,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=20, r=200, t=100, b=50),  # Increased right margin for text
        xaxis=dict(
            showline=True,
            linewidth=2,
            linecolor='black',
            showgrid=False,
            zeroline=False,
            tickfont=dict(family="Palatino", size=22)
        ),
        yaxis=dict(
            showline=True,
            linewidth=2,
            linecolor='black',
            showgrid=False,
            tickfont=dict(family="Palatino", size=22),
            title=None  # Remove y-axis label
        ),
    )
    
    # Remove the border
    fig.update_layout(
        shapes=[
            dict(
                type='rect',
                xref='paper',
                yref='paper',
                x0=0,
                y0=0,
                x1=1,
                y1=1,
                line=dict(color="white", width=2),
            )
        ]
    )

    # Save figure as PDF
    fig.write_image(f"../results/{column_name}_bar_plot.pdf")
    
    return fig

# Example usage:
fig = create_horizontal_bar_plot(all_data, 'multipaper_sentences_ndcg@100')
fig.show()